In [1]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

In [2]:
# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

In [4]:
# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]

In [6]:
# Given order
order = [
    "Актуализация", "Внебаланс (new)", "ДИЗ", "ДИЗ (возражения)", "ДИЗ (чередование)",
    "Коллекшн ранний сбор", "Матрица мотиваторов", "Матрица мотиваторов (распознавание эмоций)",
    "Матрица мотиваторов Хард. Ранний сбор", "Матрица мотиваторов Хард. Ранний сбор (кредитные карты)",
    "Матрица мотивации. Выезд", "Мультидоговоры. Универсальный", "Неконтактные", "Неконтактные (внебаланс)",
    "Неконтактные (поздний сбор)", "Неконтактные Суд", "Ожидание выезда", "Ожидание суда",
    "Подготовка к передаче в КА", "Преколлекшн (rename)", "Суд", "Усиление ПС Предцессия", 
    "Усиление позднего сбора", "ФЗ-230"
]
# Given column order
column_order = [
    "АО: Абонент не отвечает", "АО: Абонент недоступен", "АО: Номер не существует", 
    "АО: Нужен внутренний номер", "АО: Соединение установлено", "АО: Умные голосовые помощники", 
    "АО: Факс", "Должник молчит", "Должник не уверен", "Должник неизвестен", "Должник умер", 
    "Запрос реструктуризации", "Заявление о факте платежа", "Клиент болен", "Не передадут информацию", 
    "Не пройдена верификация", "Неконструктивный диалог", "Нужна помощь оператора", "Нужна помощь оператора *", 
    "Обещание оплатить", "Обещание частичной оплаты", "Оплата по испол.листу", "Отказ от верификации", 
    "Отказ от оплаты", "Отрицает долг", "Перезвонить", "Просьба передать информацию", "Сброс звонка роботом", 
    "Связь прервалась"
]
# 1. Filter the dataframe
filtered_df = df[df['Результат автооценки'] != 100] # !!!
#filtered_df = df # !!!
# 2. Create the pivot table
pivot_table = filtered_df.pivot_table(index='Имя колл-листа', 
                                      columns='Результат робота', 
                                      values='Результат автооценки', 
                                      aggfunc='size', 
                                      fill_value=0)

'''pivot_table = filtered_df.pivot_table(index='Имя колл-листа', 
                                      columns='Результат робота', 
                                      values='Результат автооценки', 
                                      aggfunc='count', 
                                      fill_value=0)'''

pivot_table_mean = df.pivot_table(index='Имя колл-листа', columns='Результат робота', values='Результат автооценки', aggfunc='mean')


# 3. Ensure all desired columns are present
for col in column_order:
    if col not in pivot_table.columns:
        pivot_table[col] = 0

# Ensure all desired columns are present in the mean pivot table
for col in column_order:
    if col not in pivot_table_mean.columns:
        pivot_table_mean[col] = 0


# Reorder columns based on the given order
pivot_table = pivot_table[column_order]
pivot_table_mean = pivot_table_mean[column_order]
# Assuming pivot_table is the pivot table you previously created
pivot_table = pivot_table[column_order]
pivot_table = pivot_table.reindex(order)
pivot_table_mean = pivot_table_mean.reindex(order)

In [50]:
pivot_table

Результат робота,АО: Абонент не отвечает,АО: Абонент недоступен,АО: Номер не существует,АО: Нужен внутренний номер,АО: Соединение установлено,АО: Умные голосовые помощники,АО: Факс,Должник молчит,Должник не уверен,Должник неизвестен,...,Обещание оплатить,Обещание частичной оплаты,Оплата по испол.листу,Отказ от верификации,Отказ от оплаты,Отрицает долг,Перезвонить,Просьба передать информацию,Сброс звонка роботом,Связь прервалась
Имя колл-листа,,,,,,,,,,,,,,,,,,,,,
Актуализация,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0
Внебаланс (new),0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,56.0,...,80.0,29.0,6.0,9.0,144.0,5.0,14.0,90.0,0.0,0.0
ДИЗ,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,13.0,...,9.0,0.0,4.0,8.0,28.0,2.0,5.0,12.0,0.0,1.0
ДИЗ (возражения),0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,...,1.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0
ДИЗ (чередование),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
Коллекшн ранний сбор,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,14.0,...,52.0,0.0,0.0,19.0,16.0,2.0,4.0,30.0,0.0,4.0
Матрица мотиваторов,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,27.0,...,1665.0,0.0,0.0,29.0,460.0,50.0,19.0,110.0,0.0,3.0
Матрица мотиваторов (распознавание эмоций),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
Матрица мотиваторов Хард. Ранний сбор,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [48]:
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill
import xlsxwriter


def format_xlsx(pivot_table:pd.core.frame.DataFrame, sheet:str = 'Отчет 1', name:str = "pivot_table_gradient_colorscale.xlsx"):
    # Create an Excel writer and export the pivot table to an Excel file
    excel_file_path = name
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        pivot_table.to_excel(writer, sheet_name=sheet, index=True)

        # Get the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets[sheet]

        # Get the dimensions of the pivot table
        max_row = len(pivot_table)
        max_col = len(pivot_table.columns)

        # Add a format for the header cells
        header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'border': 1, 'bg_color': '#EFEFEF', 'align': 'center'})

        # Set the column width and format for the header
        for col_num, value in enumerate(pivot_table.columns.values):
            worksheet.write(0, col_num + 1, value, header_format)
            column_len = max(pivot_table[value].astype(str).str.len().max(), len(value)) + 2
            worksheet.set_column(col_num + 1, col_num + 1, column_len)
        
        # Add a format for the value cells
        value_format = workbook.add_format({'border': None, 'valign': 'vcenter', 'align': 'center',})

        # Get Max and Min of the value FFS
        max_value = pivot_table.max().max()
        min_value = pivot_table.min().min()


        # Apply gradient color scale to value cells
        worksheet.conditional_format(1, 1, max_row, max_col, {
            'type': '3_color_scale',
            'min_color': '#A6D86E',  # Green
            'mid_color': '#FFFFFE',  # White (for NaN)
            'max_color': '#e85f5f',  # Red
            'min_type': 'num',
            'mid_type': 'num',
            'max_type': 'num'
        })
        print(f'file: {name} -- Transformed 0')

In [49]:
format_xlsx(pivot_table)

file: pivot_table_gradient_colorscale.xlsx -- Transformed 0
